# Summary

Adapter model training and evaluation modeled after Don't Stop Pretraining paper

# Setup

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install datasets
!pip install huggingface_hub
!pip install scikit-learn
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
!pip install -qq adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.0/256.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 39.0 MB/s eta 0:00:00


In [4]:
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)
from sklearn.metrics import accuracy_score,  f1_score

from datasets import load_dataset

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Create function for metrics for evaluation

In [5]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

Load the tokenizer - Roberta Base

In [6]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")#, truncation=True, padding=True)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [25]:
tokenizer.model_max_length

512

In [26]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained("roberta-base")

# Training

## DAPT

Continue pretraining ROBERTA on a large corpus of unlabeled
domain-specific text (Reviews dataset).

In [8]:
from datasets import load_dataset, DatasetDict

# split = 'train[0:20%]' # reduce the working size to speed up iteration

# dataset_train = load_dataset("BigTMiami/amazon_25M_reviews_condensed", split='train[0:20%]')
# dataset_valid = load_dataset("BigTMiami/amazon_25M_reviews_condensed", split='dev[0:20%]')
# dataset_test = load_dataset("BigTMiami/amazon_25M_reviews_condensed", split='test[0:20%]')

domain_dataset = load_dataset("BigTMiami/amazon_split_25M_reviews_20_percent_condensed")

# # 90% train, 10% test + validation
# train_testvalid = full_dataset.train_test_split(test_size=0.1)

# # Split the 10% test + valid in half test, half valid
# test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

# # gather everyone if you want to have a single DatasetDict
# domain_dataset = DatasetDict({'train': train_testvalid['train'], 'test': test_valid['test'], 'validation': test_valid['train']})

Generating train split:   0%|          | 0/862683 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8360 [00:00<?, ? examples/s]

In [25]:
domain_dataset = DatasetDict({'train': domain_dataset['train'], 'validation': domain_dataset['validation']})

In [11]:
# Process domain dataset to be batch size
domain_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 862683
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8360
    })
})

In [32]:
domain_dataset['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 862683
})

In [28]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")#, truncation=True, padding=True)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [29]:
from adapters import AutoAdapterModel
# Model
dapt_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

# Add a new adapter to model - use seq bn architecture
dapt_model.add_adapter("review_adapter", config="seq_bn")

# Add a matching classification head
dapt_model.add_classification_head(
    "review_adapter",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )

# Activate the adapter
dapt_model.train_adapter("review_adapter")
dapt_model.set_active_adapters("review_adapter")

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
# Training
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    output_dir="./adapter_dapt_reviews",
    learning_rate=0.0005,
    adam_epsilon=1e-6,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=128, # 16*128*6
    # logging_steps=200,
    adam_beta1 = 0.9,
    adam_beta2 = 0.98,
    weight_decay = 0.01,
    warmup_ratio = 0.06,
    # masking_probability=0.15,
    lr_scheduler_type="linear",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

dapt_trainer = AdapterTrainer(
    model=dapt_model,
    args=training_args,
    # tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=domain_dataset["train"],
    eval_dataset=domain_dataset["validation"],
    compute_metrics=compute_accuracy,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [31]:
# TRAINING
dapt_trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


TypeError: 'dict' object is not callable

In [ ]:
dapt_trainer.evaluate()

In [ ]:
dapt_trainer.push_to_hub("RobertaAdapter_reviews_DAPT_v1")

TAPT

In [30]:
task_dataset = load_dataset("BigTMiami/amazon_helpfulness")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

In [42]:
task_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 115251
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    dev: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [31]:
task_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=classification_config,
)

# Add a new adapter to model
task_model.add_adapter("task_review_helpfulness_adapter", config="seq_bn")

# Add a matching classification head
task_model.add_classification_head(
    "reviews_helpfulness_adapter",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )

# Activate the adapter
task_model.train_adapter("task_review_helpfulness_adapter")

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'heads.default.3.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
# Use Adapter Training Class
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=0.0001,
    adam_epsilon=1e-6,
    num_train_epochs=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=128, # notes on pretraining
    #logging_steps=200,
    adam_beta1 = 0.9,
    adam_beta2 = 0.98,
    weight_decay = 0.01,
    warmup_ratio = 0.06,
    # masking_probability=0.15,
    lr_scheduler_type="linear",
    output_dir="./adapter_tapt_helpfulness",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

task_trainer = AdapterTrainer(
    model=task_model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=task_dataset["train"],
    eval_dataset=task_dataset["dev"],
    compute_metrics=compute_accuracy,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [33]:
# TRAINING
task_trainer.train()

ValueError: Expected input batch_size (16) to match target batch_size (8192).

In [ ]:
# Evaluate
task_trainer.evaluate()

In [ ]:
task_trainer.push_to_hub("RobertaAdapter_helpfulness_TAPT")

In [ ]:
# Save adapter to hugging face
task_trainer.save_model("RobertaAdapter_helpfulnessModel")

# Evaluation

In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=task_model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

In [ ]:
# Save trained adapter
task_model.save_adapter("./test_adapter1", "rotten_tomatoes")

!ls -lh final_adapter